# ПРАКТИЧЕСКАЯ РАБОТА № 2 (Вариант 8)
***
## Расчет условной энтропии дискретных сообщений, передаваемых по каналу связи с помехами
---

**Цель работы:** приобрести умение определять условную энтропию, энтропию объединения и информационные потери для дискретных сообщений, передаваемых по каналу связи, в котором присутствуют помехи.

### Ход работы:
---

#### **1.** Зададим частоты появления символов на выходе канала, число случаев передачи и вероятности появления этих символов

In [2]:
# Число случаев передачи
count = 150

# Частоты появления символов
s = {
    1: [148, 0, 2, 0],
    2: [0, 147, 1, 2],
    3: [1, 0, 149, 0],
    4: [0, 3, 1, 146]
    }

ps = {
    1: 0.40,
    2: 0.31,
    3: 0.19,
    4: 0.10
}

In [ ]:
## Вывод на экран
from IPython.display import display, Markdown

ps_md = r'$\begin{array}{lcl}' + f"p(s_1)={ps[1]} \\\\ p(s_2)={ps[2]} \\\\ p(s_3)={ps[3]} \\\\ p(s_4)={ps[4]}" + r' \end{array}$'

output = f"""
Пусть при передаче сообщений по каналу связи с шумами была получена следующая статистика: 
при передаче $s_1$: символ $s_1$ из {count} раз был принят {s[1][0]} раз, {s[1][2]} раза был принят символ $s_3$;
при передаче $s_2$: {s[2][1]} раз был принят $s_2$, {s[2][2]} раза – $s_3$ и {s[2][3]} раз – символ $s_4$; 
при передаче $s_3$: {s[3][0]} раз был принят $s_1$, и {s[3][2]} раза – $s_3$;
при передаче $s_4$ {s[4][1]} раз был принят $s_2$, {s[4][2]} раз – $s_3$ и {s[4][3]} раза – $s_4$;. Вероятности появления этих сим-
волов в передаваемых сообщениях соответственно равны:

{ps_md}
"""
display(Markdown(output))

**Канальная матрица будет иметь следующий вид:**

In [3]:
def gen_matrix(m: dict, count):
    keys = m.keys()
    x = [m[i] for i in keys]
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = round((x[i][j] / count), 3)
    return x

matrix = gen_matrix(s, count)
print(matrix)

[[0.987, 0.0, 0.013, 0.0], [0.0, 0.98, 0.007, 0.013], [0.007, 0.0, 0.993, 0.0], [0.0, 0.02, 0.007, 0.973]]


In [ ]:
## Вывод на экран
from static import MD
output = r"$$P(a|b) = " + MD.matrix(matrix) + r"$$"
display(Markdown(output))

#### **2.** Определим общую условную энтропию сообщений:

$$H(B|A) = - \sum_{i=1}^{m}\sum_{j=1}^{m}p(a_i) \cdot p(b_j|a_i) \cdot \log_2{ (p(b_j|a_i))}$$

In [4]:
from math import log2 as log

def calculate_u_entropy(m: list[list], p: dict) -> list[list]:
    H = 0
    for i in range(len(m)):
        first_part = ps[i+1]
        second_part = 0
        for j in range(len(m[i])):
            if m[i][j] == 0:
                continue
            second_part += (m[i][j] * log(m[i][j]))
        H += first_part * second_part
    return H

u_H = calculate_u_entropy(matrix, ps)
print(u_H)

0.121


In [ ]:
## Вывод на экран
from static import print_entropy
output = f"$${print_entropy(matrix, ps)}{str(round(u_H, 3))} \\space бит/символ.$$"
display(Markdown(output))

#### **3.** Определим энтропию принятых сообщений:

$$H(B)=-\sum_{j-1}^m p(b_j)\cdot\log_2(p(b_j))$$
где $$p(b_j)=-\sum_{i=1}^m p(a_i)p(b_j|a_i)$$; причём $$\sum_{j=1}^m p(b_j)=1$$

In [7]:
## Вычислим p(b_j)
def calculate_pb(m: list[list], p: dict):
    m = [list(row) for row in zip(*m)]
    pb = []
    for i in range(len(m)):
        pbj = []
        for j in range(len(m[i])):
            if m[i][j] == 0:
                continue
            pbj.append(p[j+1] * m[i][j])
        pb.append(sum(pbj))
    return pb

pb = calculate_pb(matrix, ps)

In [ ]:
## Вывод на экран
display(Markdown(MD.pb(matrix, ps, pb)))

In [9]:
## Вычислим энтропию принятых сообщений
def calculate_accepted_entropy(pbj: list):
    out = 0
    for j in range(len(pbj)):
        out += pbj[j] * log(pbj[j])
    return -out

a_H = calculate_accepted_entropy(pb)

In [ ]:
## Вывод на экран
from static import print_accepted_entropy

display(Markdown(f"$${print_accepted_entropy(pb)} {str(round(a_H, 3))} \\spaceбит /символ$$"))

#### **4.** Составим матрицу совместных вероятностей, определив для этого все совместные вероятности

$$p(a_i, b_j) = p(a_i) \cdot p(b_j|a_i)$$

In [11]:
## Вычислим матрицу совместных вероятностей
def calculate_pm(m: list[list], ps: dict) -> list[list]:
    out = []
    for i in range(len(m)):
        row = []
        for j in range(len(m[i])):
            row.append(ps[i+1] * m[i][j])
        out.append(row)
    return out

p_matrix = calculate_pm(matrix, ps)

In [ ]:
## Вывод на экран
display(Markdown(MD.pm(matrix, ps, p_matrix)))
display(Markdown(f"$$P(a,b) = {MD.matrix(p_matrix)}$$"))

#### *5.* Определим энтропию объединения

$$H(A,B) = -\sum_{i=1}^m \sum_{j=1}^m p(a_i, b_j) \cdot \log_2(p(a_i, b_j))$$

In [13]:
def calculate_final_entropy(m: list[list]) -> list[list]:
    H = 0
    for i in range(len(m)):
        for j in range(len(m[i])):
            if m[i][j] == 0:
                continue
            H += (m[i][j] * log(m[i][j]))
    return -H

H = calculate_final_entropy(p_matrix)

In [ ]:
## Вывод на экран
from static import print_final_entropy
display(Markdown(f"$${print_final_entropy(p_matrix)}{str(round(H, 4))}$$"))

#### **6.** Найдем информационные потери при передаче 500 символов

$$\Delta I = n \cdot H(A|B);$$
$$H(A|B)=H(A,B)-H(B)$$

In [15]:
H_a_b = H - a_H

In [ ]:
display(Markdown(f"$$H(A|B) = {round(H, 4)} - {round(a_H, 4)} = {round(H_a_b, 4)} \\space бит/символ$$"))

In [17]:
count = 500
delta_I = count * H_a_b 

In [ ]:
display(Markdown(f"$$\\Delta I = {count} \\cdot {round(H_a_b, 4)} = {round(delta_I, 1)} \\space бит.$$"))

#### *7.* Найдём количество принятой информации

$$I=n \cdot H(B) - \Delta I$$

In [19]:
I = (count * a_H) - delta_I

In [ ]:
display(Markdown(f"$$I = {count} \\cdot {round(a_H, 4)} - {round(delta_I, 1)} = {round(I,1)} \\space бит.$$"))